# Vector Database Basics

Vector databases help us store, manage, and query the embeddings we created for generative AI, recommenders, and search engines.

Across many of the common use cases, users often find that they need to manage more than just vectors.
To make it easier for practitioners, vector databases should store and manage all of the data they need:
- embedding vectors
- categorical metadata
- numerical metadata
- timeseries metadata
- text / pdf / images / video / point clouds

And support a wide range of query workloads:
- Vector search (may require ANN-index)
- Keyword search (requires full text search index)
- SQL (for filtering)

For this exercise we'll use LanceDB since it's open source and easy to setup

In [ ]:
# pip install -U --quiet lancedb pandas pydantic [This has been pre-installed for you]

## Creating tables and adding data

Let's create a LanceDB table called `cats_and_dogs` under the local database directory `~/.lancedb`.
This table should have 4 fields:
- the embedding vector
- a string field indicating the species (either "cat" or "dog")
- the breed
- average weight in pounds

We're going to use pydantic to make this easier. First let's create a pydantic model with those fields

In [1]:
from lancedb.pydantic import vector, LanceModel

class CatsAndDogs(LanceModel):
    vector: vector(2)
    species: str
    breed: str
    weight: float

Now connect to a local db at ~/.lancedb and create an empty LanceDB table called "cats_and_dogs"

In [3]:
import lancedb

db = lancedb.connect("~/.lancedb")
table_name = "cats_and_dogs"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, schema=CatsAndDogs)

Let's add some data

First some cats

In [4]:
data = [
    CatsAndDogs(
        vector=[1., 0.],
        species="cat",
        breed="shorthair",
        weight=12.,
    ),
    CatsAndDogs(
        vector=[-1., 0.],
        species="cat",
        breed="himalayan",
        weight=9.5,
    ),
]

Now call the `LanceTable.add` API to insert these two records into the table

In [5]:
table.add([dict(d) for d in data])

Let's preview the data

In [6]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


Now let's add some dogs

In [7]:
data = [
    CatsAndDogs(
        vector=[0., 10.],
        species="dog",
        breed="samoyed",
        weight=47.5,
    ),
    CatsAndDogs(
        vector=[0, -1.],
        species="dog",
        breed="corgi",
        weight=26.,
    )
]

In [8]:
table.add([dict(d) for d in data])

In [9]:
table.head().to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5
2,"[0.0, 10.0]",dog,samoyed,47.5
3,"[0.0, -1.0]",dog,corgi,26.0


## Querying tables

Vector databases allow us to retrieve data for generative AI applications. Let's see how that's done.

Let's say we have a new animal that has embedding [10.5, 10.], what would you expect the most similar animal will be?
Can you use the table we created above to answer the question?

**HINT** you'll need to use the `search` API for LanceTable and `limit` / `to_df` APIs. For examples you can refer to [LanceDB documentation](https://lancedb.github.io/lancedb/basic/#how-to-search-for-approximate-nearest-neighbors).

In [10]:
table.search([10.5, 10.,]).limit(1).to_df()

,vector,species,breed,weight,_distance
0,"[0.0, 10.0]",dog,samoyed,47.5,110.25


Now what if we use cosine distance instead? Would you expect that we get the same answer? Why or why not?

**HINT** you can add a call to `metric` in the call chain

In [11]:
table.search([10.5, 10.]).metric("cosine").limit(1).to_df()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,0.275862


## Filtering tables

In practice, we often need to specify more than just a search vector for good quality retrieval. Oftentimes we need to filter the metadata as well.

Please write code to retrieve two most similar examples to the embedding [10.5, 10.] but only show the results that is a cat.

In [12]:
table.search([10.5, 10.,]).limit(2).where("species='cat'").to_df()

,vector,species,breed,weight,_distance
0,"[1.0, 0.0]",cat,shorthair,12.0,190.25


## Creating ANN indices

For larger tables (e.g., >1M rows), searching through all of the vectors becomes quite slow. Here is where the Approximate Nearest Neighbor (ANN) index comes into play. While there are many different ANN indexing algorithms, they all have the same purpose - to drastically limit the search space as much as possible while losing as little accuracy as possible

For this problem we will create an ANN index on a LanceDB table and see how that impacts performance

### First let's create some data

Given the constraints of the classroom workspace, we'll complete this exercise by creating 100,000 vectors with 16D in a new table. Here the embedding values don't matter, so we simply generate random embeddings as a 2D numpy array. We then use the vec_to_table function to convert that in to an Arrow table, which can then be added to the table.

In [16]:
from lance.vector import vec_to_table
import numpy as np

mat = np.random.randn(1_000_000, 128)
table_name = "exercise3_ann"
db.drop_table(table_name, ignore_missing=True)
table = db.create_table(table_name, vec_to_table(mat))

### Let's establish a baseline without an index

Before we create the index, let's make sure know what we need to compare against.

We'll generate a random query vector and record it's value in the `query` variable so we can use the same query vector with and without the ANN index.

In [18]:
query = np.random.randn(128)
table.search(query).limit(10).to_df()

,vector,_distance
0,"[-0.4427774, 1.1762794, 0.90885633, -0.2070089...",130.966064
1,"[0.6356288, 0.14960878, -0.22711003, -0.202752...",133.387665
2,"[2.0203872, -0.9185021, -0.44001573, -0.110723...",133.528336
3,"[-0.53038704, 0.07763474, -1.1231655, 0.986622...",134.497299
4,"[-0.11717089, 2.290166, 0.8622206, 1.2124386, ...",134.531647
5,"[1.6600263, -0.2658385, -0.25331306, 1.169905,...",134.844208
6,"[0.33511317, -0.3896364, -0.6377718, 0.0585895...",135.966003
7,"[0.24752267, 0.7009984, -0.37407437, 0.0385392...",135.973434
8,"[0.95187956, 0.22050862, 0.44407365, 1.1771716...",137.600830
9,"[1.6078576, 0.23311362, -0.010952189, 0.228119...",138.415237


Please write code to compute the average latency of this query

In [20]:
%timeit table.search(np.random.randn(128)).limit(10).to_arrow();

358 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Now let's create an index

There are many possible index types ranging from hash based to tree based to partition based to graph based.
For this task, we'll create an IVFPQ index (partition-based index with product quantization compression) using LanceDB.

Please create an IVFPQ index on the LanceDB table such that each partition is 4000 rows and each PQ subvector is 8D.

**HINT** 
1. Total vectors / number of partitions = number of vectors in each partition
2. Total dimensions / number of subvectors = number of dimensions in each subvector
3. This step can take about 7-10 minutes to process and execute in the classroom workspace.

In [21]:
table.create_index(num_partitions=256, num_sub_vectors=16)

Now let's search through the data again. Notice how the answers now appear different.
This is because an ANN index is always a tradeoff between latency and accuracy.

In [22]:
table.search(query).limit(10).to_df()

,vector,_distance
0,"[-1.2070754, 0.6254262, 0.4951973, 0.9828658, ...",110.504936
1,"[-0.27480194, 0.21665353, 0.16625892, 0.876036...",122.948448
2,"[1.4384358, 0.56784505, 0.25393167, 0.08993257...",123.030777
3,"[0.07943219, 1.3375413, -0.3963346, 0.06292188...",123.459213
4,"[0.2868396, 0.66035104, -0.44687456, -0.955964...",125.227531
5,"[1.4590665, -0.82102317, -0.64377624, -0.73783...",125.823395
6,"[-0.41693315, 0.45160192, -0.67942655, -0.6835...",126.457962
7,"[0.689404, 0.85233796, -0.08644608, -0.0190996...",127.737000
8,"[-0.2621002, 1.3222489, 0.92640716, 1.5544025,...",128.001877
9,"[0.65935326, -0.011864699, 0.66687346, 0.06414...",128.327927


Now write code to compute the average latency for querying the same table using the ANN index.

**SOLUTION** The index is implementation detail, so it should just be running the same code as above. You should see almost an order of magnitude speed-up. On larger datasets, this performance difference should be even more pronounced.

In [23]:
%timeit table.search(np.random.randn(128)).limit(10).to_arrow();

12.9 ms ± 53.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Deleting rows

Like with other kinds of databases, you should be able to remove rows from the table.
Let's go back to our tables of cats and dogs

In [24]:
table = db["cats_and_dogs"]

In [25]:
len(table)

4

Can you use the `delete` API to remove all of the cats from the table?

**HINT** use a SQL like filter string to specify which rows to delete from the table

In [26]:
table.delete("species='cat'")

In [27]:
len(table)

2

## What if I messed up?

Errors is a common occurrence in AI. What's hard about errors in vector search is that oftentimes a bad vector doesn't cause a crash but just creates non-sensical answers. So to be able to rollback the state of the database is very important for debugging and reproducibility

So far we've accumulated 4 actions on the table:
1. creation of the table
2. added cats
3. added dogs
4. deleted cats

What if you realized that you should have deleted the dogs instead of the cats?

Here we can see the 4 versions that correspond to the 4 actions we've done

In [28]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2025, 10, 31, 22, 40, 37, 416839),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2025, 10, 31, 22, 43, 2, 834648),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2025, 10, 31, 22, 43, 43, 409664),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2025, 10, 31, 23, 10, 36, 588554),
  'metadata': {}}]

Please write code to restore the version still containing the whole dataset

In [29]:
table = db["cats_and_dogs"]

In [30]:
len(table)

2

In [31]:
# restore to version 3

table.restore(3)

In [32]:
# delete the dogs instead

table.delete("species='dog'")

In [33]:
table.list_versions()

[{'version': 1,
  'timestamp': datetime.datetime(2025, 10, 31, 22, 40, 37, 416839),
  'metadata': {}},
 {'version': 2,
  'timestamp': datetime.datetime(2025, 10, 31, 22, 43, 2, 834648),
  'metadata': {}},
 {'version': 3,
  'timestamp': datetime.datetime(2025, 10, 31, 22, 43, 43, 409664),
  'metadata': {}},
 {'version': 4,
  'timestamp': datetime.datetime(2025, 10, 31, 23, 10, 36, 588554),
  'metadata': {}},
 {'version': 5,
  'timestamp': datetime.datetime(2025, 10, 31, 23, 10, 36, 619977),
  'metadata': {}},
 {'version': 6,
  'timestamp': datetime.datetime(2025, 10, 31, 23, 10, 36, 629037),
  'metadata': {}}]

In [34]:
table.to_pandas()

,vector,species,breed,weight
0,"[1.0, 0.0]",cat,shorthair,12.0
1,"[-1.0, 0.0]",cat,himalayan,9.5


## Dropping a table

You can also choose to drop a table, which also completely removes the data.
Note that this operation is not reversible.

In [35]:
"cats_and_dogs" in db

True

Write code to irrevocably remove the table "cats_and_dogs" from the database

In [36]:
db.drop_table("cats_and_dogs")

How would you verify that the table has indeed been deleted?

In [37]:
table.name in db

False

## Summary

Congrats, in this exercise you've learned the basic operations of vector databases from creating tables, to adding data, and to querying the data. You've learned how to create indices and you saw first hand how it changes the performance and the accuracy. Lastly, you've learned how to debug and rollback when errors happen.